In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 33146608
paper_name = 'kintaka_moriya_2020' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
sga_results_files = ['elife-54080-fig2-data1-v2.xlsx','Data_S3_tGFP_SGA_raw data.xlsx','Data_S4_NES-tGFP_SGA_raw data.xlsx']

In [6]:
sheet_names = [['DMA U Rep1','DMA U Rep2'],['DMA LU Rep1','DMA LU Rep2']]
original_data_list = []

In [7]:
dataset_ids = [16674,16675,16681,16677,16682,16678,16683,16685,16684]

In [8]:
num_dataset = 0
for f in sga_results_files:
    print('File %s' % f)
    for sheet_names_this in sheet_names:
        this_data = pd.DataFrame()
        for s in sheet_names_this:
            print('Sheet %s' % s)
            t = pd.read_excel('raw_data/' + f, sheet_name=s)
            print('Original data dimensions: %d x %d' % (t.shape))
            t = t[['Array ORF','Score']]
            t['Score'] = t['Score'].astype(float)
            this_data = pd.concat((this_data, t), axis=0)
        this_data.columns = ['orf', dataset_ids[num_dataset]]
        num_dataset += 1
        original_data_list.append(this_data)

File elife-54080-fig2-data1-v2.xlsx
Sheet DMA U Rep1
Original data dimensions: 4322 x 11
Sheet DMA U Rep2
Original data dimensions: 4322 x 11
Sheet DMA LU Rep1
Original data dimensions: 4322 x 11
Sheet DMA LU Rep2
Original data dimensions: 4322 x 11
File Data_S3_tGFP_SGA_raw data.xlsx
Sheet DMA U Rep1
Original data dimensions: 4322 x 11
Sheet DMA U Rep2
Original data dimensions: 4322 x 11
Sheet DMA LU Rep1
Original data dimensions: 4322 x 11
Sheet DMA LU Rep2
Original data dimensions: 4322 x 11
File Data_S4_NES-tGFP_SGA_raw data.xlsx
Sheet DMA U Rep1
Original data dimensions: 4322 x 11
Sheet DMA U Rep2
Original data dimensions: 4322 x 11
Sheet DMA LU Rep1
Original data dimensions: 4322 x 11
Sheet DMA LU Rep2
Original data dimensions: 4322 x 11


In [9]:
gfp_expression_file = 'elife-54080-fig4-data1-v2.xlsx'
sheet_names = ['GFPunit (GFP)','GFPunit(NES-tGFP)','GFPunit(tGFP)']

In [10]:
for s in sheet_names:
    t = pd.read_excel('raw_data/' + gfp_expression_file, sheet_name=s)
    print('Original data dimensions: %d x %d' % (t.shape))
    t = t.loc[t['array name']=='DMA']
    t = t[['ORF','GFPunit_Average']]
    t['GFPunit_Average'] = t['GFPunit_Average'].astype(float)
    t.columns = ['orf', dataset_ids[num_dataset]]
    num_dataset += 1
    original_data_list.append(t)

Original data dimensions: 4989 x 11
Original data dimensions: 5095 x 11
Original data dimensions: 5095 x 11


In [11]:
len(original_data_list)

9

In [12]:
data_list = []
for df in original_data_list:
    df['orf'] = df['orf'].astype(str)
    df['orf'] = clean_orf(df['orf'])
    df = df.groupby('orf').mean().reset_index()
    df['orf'] = translate_sc(df['orf'], to='orf')
    t = looks_like_orf(df['orf'])
    print(df.loc[~t,])
    df = df.groupby('orf').mean()
    data_list.append(df)

Empty DataFrame
Columns: [orf, 16674]
Index: []
Empty DataFrame
Columns: [orf, 16675]
Index: []
Empty DataFrame
Columns: [orf, 16681]
Index: []
Empty DataFrame
Columns: [orf, 16677]
Index: []
Empty DataFrame
Columns: [orf, 16682]
Index: []
Empty DataFrame
Columns: [orf, 16678]
Index: []
Empty DataFrame
Columns: [orf, 16683]
Index: []
Empty DataFrame
Columns: [orf, 16685]
Index: []
Empty DataFrame
Columns: [orf, 16684]
Index: []


In [13]:
data = pd.concat(data_list, axis=1, join='outer')

In [14]:
data.shape

(4268, 9)

In [15]:
data.index.name='orf'

# Prepare the final dataset

In [16]:
datasets = datasets.reindex(index=dataset_ids)

In [17]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [18]:
data.head()

dataset_id,16674,16675,16681,16677,16682,16678,16683,16685,16684
data_type,value,value,value,value,value,value,value,value,value
orf,,,,,,,,,
YAL002W,-0.158555,-0.445345,-0.107395,-0.180620,-0.070805,-0.222020,0.765489,0.85,0.80
YAL004W,0.040180,-0.074145,-0.059960,-0.125135,0.018800,-0.087520,1.049857,0.85,1.05
YAL005C,0.020125,0.001315,0.006005,-0.016695,-0.017935,-0.079770,1.086268,0.95,1.00
YAL007C,-0.080920,-0.015805,-0.075265,0.005355,-0.032655,-0.003115,1.169982,1.15,1.10
YAL008W,-0.027970,0.026235,0.004385,0.033825,0.056235,0.018810,1.206636,1.60,1.30


## Subset to the genes currently in SGD

In [19]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [20]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

In [21]:
data.head()

,dataset_id,16674,16675,16681,16677,16682,16678,16683,16685,16684
,data_type,value,value,value,value,value,value,value,value,value
gene_id,orf,,,,,,,,,
2,YAL002W,-0.158555,-0.445345,-0.107395,-0.180620,-0.070805,-0.222020,0.765489,0.85,0.80
1863,YAL004W,0.040180,-0.074145,-0.059960,-0.125135,0.018800,-0.087520,1.049857,0.85,1.05
4,YAL005C,0.020125,0.001315,0.006005,-0.016695,-0.017935,-0.079770,1.086268,0.95,1.00
5,YAL007C,-0.080920,-0.015805,-0.075265,0.005355,-0.032655,-0.003115,1.169982,1.15,1.10
6,YAL008W,-0.027970,0.026235,0.004385,0.033825,0.056235,0.018810,1.206636,1.60,1.30


# Normalize

In [22]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [23]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [24]:
data_norm[data.isnull()] = np.nan

In [25]:
data_all = data.join(data_norm)

In [26]:
data_all.head()


,dataset_id,16674,16675,16681,16677,16682,16678,16683,16685,16684,16674,16675,16681,16677,16682,16678,16683,16685,16684
,data_type,value,value,value,value,value,value,value,value,value,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,,,,,,,,,
2,YAL002W,-0.158555,-0.445345,-0.107395,-0.180620,-0.070805,-0.222020,0.765489,0.85,0.80,-2.400198,-5.249360,-1.943343,-2.755363,-1.082138,-2.691196,-1.779535,-0.902991,-1.231421
1863,YAL004W,0.040180,-0.074145,-0.059960,-0.125135,0.018800,-0.087520,1.049857,0.85,1.05,0.620518,-0.785642,-1.056426,-1.917299,0.315063,-1.013373,-0.024446,-0.902991,-0.144873
4,YAL005C,0.020125,0.001315,0.006005,-0.016695,-0.017935,-0.079770,1.086268,0.95,1.00,0.315687,0.121772,0.176955,-0.279385,-0.257742,-0.916696,0.200276,-0.520138,-0.362183
5,YAL007C,-0.080920,-0.015805,-0.075265,0.005355,-0.032655,-0.003115,1.169982,1.15,1.10,-1.220168,-0.084097,-1.342592,0.053666,-0.487269,0.039539,0.716948,0.245567,0.072437
6,YAL008W,-0.027970,0.026235,0.004385,0.033825,0.056235,0.018810,1.206636,1.60,1.30,-0.415343,0.421438,0.146665,0.483687,0.898783,0.313042,0.943175,1.968403,0.941675


# Print out

In [27]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [28]:
from IO.save_data_to_db3 import *

In [29]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/9 [00:00<?, ?it/s]

Deleting all datasets for PMID 33146608...
Inserting the new data...


100%|██████████| 9/9 [00:56<00:00,  6.24s/it]

Updating the data_modified_on field...
